In [ ]:
from notebooks import *
import sqlalchemy as sqla
sg.init(app=None)

In [ ]:
samples_per_mb = 512 * 1024  # (Determined empirically)
samples_per_s = 22050
mb_per_s = samples_per_s / samples_per_mb
n_xc_recs = len(xc.metadata)  # (sg.xc_meta ~ xc.metadata[downloaded == True])
n_xc_recs_us = len(xc.metadata[lambda df: df.country == 'United States'])
display(
    n_xc_recs,
    n_xc_recs_us,
)

399859

41331

In [ ]:
from humanize import naturalsize
recs = (sg.xc_meta
    .sample(n=1000, random_state=0)
    .pipe(recs_featurize_audio_meta)
    .pipe(df_inspect, lambda df: (
        '', 'n_recs',
        '  sample: %s' % len(df),
        '  us:     %s' % n_xc_recs_us,
        '  all:    %s' % n_xc_recs,
        '', 'duration_s',
        '  avg: %.1fs' % (df.duration_s.sum() / len(df)),
        '  us:  %.1fs' % (df.duration_s.sum() / len(df) * n_xc_recs_us),
        '  all: %.1fs' % (df.duration_s.sum() / len(df) * n_xc_recs),
        '', 'duration_s if clipped to ≤10s',
        '  avg: %.1fs' % (df.duration_s.map(lambda x: np.clip(x, 0, 10)).sum() / len(df)),
        '  us:  %.1fs' % (df.duration_s.map(lambda x: np.clip(x, 0, 10)).sum() / len(df) * n_xc_recs_us),
        '  all: %.1fs' % (df.duration_s.map(lambda x: np.clip(x, 0, 10)).sum() / len(df) * n_xc_recs),
        '', 'samples_mb',
        '  avg: %s' % naturalsize(1024**2 * df.samples_mb.sum() / len(df)),
        '  us:  %s' % naturalsize(1024**2 * df.samples_mb.sum() / len(df) * n_xc_recs_us),
        '  all: %s' % naturalsize(1024**2 * df.samples_mb.sum() / len(df) * n_xc_recs),
        '', 'samples_mb if clipped to ≤10s',
        '  avg: %s' % naturalsize(1024**2 * df.samples_mb.map(lambda x: np.clip(x, 0, 10 * mb_per_s)).sum() / len(df)),
        '  us:  %s' % naturalsize(1024**2 * df.samples_mb.map(lambda x: np.clip(x, 0, 10 * mb_per_s)).sum() / len(df) * n_xc_recs_us),
        '  all: %s' % naturalsize(1024**2 * df.samples_mb.map(lambda x: np.clip(x, 0, 10 * mb_per_s)).sum() / len(df) * n_xc_recs),
        # df[:10],
    ))
)

[01:53:31.982] INFO   [2/3 slower] Loading xc.metadata -> xc_raw_recs (.audio, more metadata)...

''

'n_recs'

'  sample: 976'

'  us:     41331'

...

''

'samples_mb if clipped to ≤10s'

'  avg: 418.9 kB'

'  us:  17.3 GB'

'  all: 167.5 GB'

In [ ]:
# Example: sqlite + sqla
import sqlalchemy as sqla
# An engine manages conn pools
eng = sqla.create_engine(
    # 'sqlite://',  # In mem
    # 'sqlite:///:memory:',  # Also in mem
    # 'sqlite:///foo.db',  # Rel path
    'sqlite:////tmp/foo.db',  # Abs path
)
# eng.begin() makes a conn with a txn
with eng.begin() as conn:
    conn.execute("create table if not exists x (a integer, b integer)")
    conn.execute("insert into x (a, b) values (1, 1)")
    conn.execute("insert into x (a, b) values (2, 2)")
# eng.dispose() releases conn pools
eng.dispose()

In [ ]:
# Example: pd -> sqlite -> pd
eng = sqla.create_engine('sqlite://')
df = DF([dict(x=1,y=2), dict(x=-1,y=3)])
df.to_sql('xs', con=eng)
pd.read_sql_table('xs', con=eng)

,index,x,y
0,0,1,2
1,1,-1,3


In [ ]:
# Measurements

def measure_parquet_size(
    df,
    compression,
    path=None,
    **kwargs,
) -> int:
    kwargs.setdefault('engine', 'fastparquet')
    kwargs['compression'] = compression
    ext = {'uncompressed': '', 'gzip': '.gz'}[compression]
    ret = {}
    path = Path(path or f'/tmp/df-{secrets.token_hex(4)}.parquet{ext}')
    ret['_path'] = str(path)
    debug_print(f'Writing: {path}')
    # total
    df.to_parquet(path, **kwargs)
    ret['_size'] = Path(path).stat().st_size
    # cols
    cols_dir = path.with_suffix('.cols')
    mkdir_p(cols_dir)
    for c in df:
        col_path = (cols_dir / c).with_suffix('.parquet')
        df[[c]].to_parquet(col_path, **kwargs)
        ret[c] = Path(col_path).stat().st_size
    return ret

def measure_sqlite_size(
    df,
    compression,
    path=None,
) -> int:
    assert compression in ['uncompressed', 'gzip']
    ext = {'uncompressed': '', 'gzip': '.gz'}[compression]
    ret = {}
    path = Path(path or f'/tmp/df-{secrets.token_hex(4)}.db{ext}')
    debug_print(f'Writing: {path}')
    ret['_path'] = str(path)
    eng = sqla.create_engine(f'sqlite:///{path}')
    try:
        df.to_sql('df', con=eng)
        with eng.begin() as conn:
            conn.execute('analyze')

        with eng.begin() as conn:
            [col_sizes] = [dict(row) for row in conn.execute('''
                select %s from df
            ''' % ', '.join(
                'sum(length(%(col)s)) as %(col)s' % dict(col=col)
                for col in df
            ))]
            ret.update(col_sizes)

    finally:
        eng.dispose()
    if compression == 'gzip':
        shell('mv %(path)s %(path)s.x && gzip %(path)s.x && mv %(path)s.x.gz %(path)s', path=path, _verbose=False)
    ret['_size'] = Path(path).stat().st_size
    return ret

def measure_sizes(
    df,
    parquet=True,
    sqlite=True,
    uncompressed=True,
    gzip=True,
    show_df=True,
):
    if show_df:
        display(df[:1].T.reset_index())
    dicts = [
        *([measure_parquet_size(df, compression='uncompressed')] if parquet and uncompressed else []),
        *([measure_parquet_size(df, compression='gzip')]         if parquet and gzip         else []),
        *([measure_sqlite_size(df, compression='uncompressed')]  if sqlite  and uncompressed else []),
        *([measure_sqlite_size(df, compression='gzip')]          if sqlite  and gzip         else []),
    ]
    debug_print('Done')
    with pd.option_context('display.precision', 8):
        display(
            DF(dicts)
            .assign(_n_rows=len(df))
            .pipe(df_col_map, _size=humanize.naturalsize)
            .set_index(['_path', '_n_rows', '_size'])
            .T
            .pipe(lambda df: df.sort_values(df.columns[0], ascending=False))
        )

# %%
df = DF(np.random.rand(1000, 3), columns=list(string.ascii_lowercase[:3]))
measure_sizes(df)

,index,0
0,a,0.260
1,b,0.204
2,c,0.921


[<ipython-input-6-6452cbb47530>:14 01:53:32.727] measure_parquet_size: Writing: /tmp/df-e06ba8f3.parquet

[<ipython-input-6-6452cbb47530>:14 01:53:33.148] measure_parquet_size: Writing: /tmp/df-f538cfad.parquet.gz

[<ipython-input-6-6452cbb47530>:35 01:53:33.170] measure_sqlite_size: Writing: /tmp/df-12beaab6.db

[<ipython-input-6-6452cbb47530>:35 01:53:33.226] measure_sqlite_size: Writing: /tmp/df-aeba9e44.db.gz

[<ipython-input-6-6452cbb47530>:72 01:53:33.311] measure_sizes: Done

_path,/tmp/df-e06ba8f3.parquet,/tmp/df-f538cfad.parquet.gz,/tmp/df-12beaab6.db,/tmp/df-aeba9e44.db.gz
_n_rows,1000,1000,1000,1000
_size,25.0 kB,23.7 kB,65.5 kB,35.9 kB
a,8424,8007,16993,16993
b,8424,8002,16994,16994
c,8424,8005,17027,17027


In [ ]:
# TODO TODO How big is the payload?

def np_to_npy_bytes(x: np.ndarray) -> bytes:
    # From https://stackoverflow.com/a/18622264/397334
    #   - np.save docs: https://docs.scipy.org/doc/numpy/reference/generated/numpy.save.html
    #   - .npy isn't slow or big: https://stackoverflow.com/a/41425878/397334
    out = io.BytesIO()
    np.save(out, x)
    out.seek(0)
    return out.read()

df = (recs

    .sample(random_state=0,
        n=2,
        # n=10,
        # n=100,
    )

    # Featurize
    #   .pipe(recs_featurize_audio_feat_spectro)
    # [INLINE]
    .pipe(sg.load.audio, scheduler='threads')
    .pipe(sg.projection.transform)  # .feat
    .assign(spectro=lambda df: sg.features.spectro(df, scheduler='threads', cache=True))  # .spectro is input for thumb/micro

    # Featurize
    #   .pipe(recs_featurize_thumb_micro,
    #       thumb_s=0,
    #       audio_s=10,
    #       scale=1,
    #       audio=False,  # Don't include <audio>
    #   )
    # [INLINE]
    .pipe(df_assign_first, **dict(
        micro=df_cell_spectros(plot_spectro_micro.many, sg.features,
            wrap_s=10,  # (= audio_s)
            audio=False,  # Don't include <audio> (from display_with_audio)
            **{  # (= plot_many_kwargs)
                'scale': dict(h=int(40 * 1)),  # (scale=1)
                'progress': dict(use='dask', scheduler='threads'),
                '_nocache': True,  # Dev: disable plot_many cache since it's blind to most of our sub-many code changes
            }
        ),
    ))

    # Featurize
    .pipe(df_map_rows, lambda row: pd.Series(dict(**row,
        audio_wav          = audio_to_bytes(row.audio.unbox, format='wav'),
        audio_128k_mp3     = audio_to_bytes(row.audio.unbox, format='mp3', bitrate='128k'),
        audio_064k_mp3     = audio_to_bytes(row.audio.unbox, format='mp3', bitrate='64k'),
        audio_032k_mp3     = audio_to_bytes(row.audio.unbox, format='mp3', bitrate='32k'),
        audio_128k_aac_mp4 = audio_to_bytes(row.audio.unbox, format='mp4', bitrate='128k', codec='aac'),
        audio_064k_aac_mp4 = audio_to_bytes(row.audio.unbox, format='mp4', bitrate='64k',  codec='aac'),
        audio_032k_aac_mp4 = audio_to_bytes(row.audio.unbox, format='mp4', bitrate='32k',  codec='aac'),
        audio_128k_fdk_mp4 = audio_to_bytes(row.audio.unbox, format='mp4', bitrate='128k', codec='libfdk_aac'),
        audio_064k_fdk_mp4 = audio_to_bytes(row.audio.unbox, format='mp4', bitrate='64k',  codec='libfdk_aac'),
        audio_032k_fdk_mp4 = audio_to_bytes(row.audio.unbox, format='mp4', bitrate='32k',  codec='libfdk_aac'),
    )))

    # Drop cols
    #   - .spectro: Trimmed to .micro/.thumb
    #   - .*_stack: Redundant
    .drop(columns=['spectro'])
    [lambda df: [c for c in df if not c.endswith('_stack')]]

    # Convert types for sqlite/parquet compat
    .pipe(df_col_map,
        background=','.join,  # List[str] -> str
        background_species=','.join,  # List[str] -> str
        micro=lambda x: ipy_formats_to_html(x),  # df_cell_display(...) -> html str
        audio=lambda x: pickle.dumps(x.unbox),  # Box(AudioSegment) -> pkl bytes
        feat=np_to_npy_bytes,  # np.ndarray -> npy bytes
    )

    # Cosmetics
    .pipe(df_reorder_cols, first=['audio', 'audio_mp3', 'audio_10s_mp3', 'micro', 'feat'])

)
measure_sizes(df,
    show_df=False,
    # parquet=False,
    # sqlite=False,
    gzip=False,  # Faster dev (gzip way slower than uncompressed)
)

[01:53:33.776] DEBUG  Load.audio:in

  len(recs): 2

  len(recs) per dataset: {xc: 2}

audio: [

] |   0% Completed |  0.0s

...

[01:53:34.257] INFO   Have:       /Users/danb/hack/bubo/features/data/cache/22050hz-1ch-16bit-mp4-libfdk_aac-32k/xc/data/PLCU/283548/audio.mp4

[<ipython-input-6-6452cbb47530>:14 01:53:34.343] measure_parquet_size: Writing: /tmp/df-f6b8b39b.parquet

[<ipython-input-6-6452cbb47530>:35 01:53:34.792] measure_sqlite_size: Writing: /tmp/df-1807e906.db

[<ipython-input-6-6452cbb47530>:72 01:53:34.923] measure_sizes: Done

_path,/tmp/df-f6b8b39b.parquet,/tmp/df-1807e906.db
_n_rows,2,2
_size,24.8 MB,8.3 MB
audio,6269869.0,2089718.0
audio_wav,6268721.0,2089330.0
audio_128k_fdk_mp4,2302724.0,767322.0
audio_128k_mp3,2280559.0,759939.0
audio_128k_aac_mp4,1927469.0,642237.0
audio_064k_aac_mp4,1163099.0,387447.0
audio_064k_fdk_mp4,1160489.0,386577.0
audio_064k_mp3,1140784.0,380014.0
